In [1]:
!pip install pytorch-ood
!pip install pandas scikit-learn pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.3/123.3 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 763.4/763.4 kB 9.2 MB/s eta 0:00:00


In [2]:
import pandas as pd
import torch
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR100, CIFAR10, MNIST, FashionMNIST

from pytorch_ood.dataset.img import (
    LSUNCrop,
    LSUNResize,
    Textures,
    TinyImageNetCrop,
    TinyImageNetResize,
    Places365,
    TinyImageNet,
)

from pytorch_ood.detector import (
    ODIN,
    Mahalanobis,
    MaxSoftmax,
)

from pytorch_ood.model import WideResNet
from pytorch_ood.utils import OODMetrics, ToUnknown, fix_random_seed

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

fix_random_seed(123)

trans = WideResNet.transform_for("cifar100-pt")
norm_std = WideResNet.norm_std_for("cifar100-pt")

In [3]:
in_dataset = CIFAR100(root="data", train=False, transform=trans, download=True)
test_datasets = [
    CIFAR100,
    CIFAR10,
    MNIST,
    FashionMNIST,
    LSUNCrop,
    LSUNResize,
    Textures,
    TinyImageNetCrop,
    TinyImageNetResize,
    Places365,
    TinyImageNet,
]
datasets = {}
for test_dataset in test_datasets:
    dataset_out_test = test_dataset(root="data", transform=trans, target_transform=ToUnknown(), download=True)
    test_loader = DataLoader(in_dataset + dataset_out_test, batch_size=256, num_workers=12)
    datasets[test_dataset.__name__] = test_loader

100%|██████████| 169001437/169001437 [00:01<00:00, 102482504.47it/s]


Extracting data/cifar-100-python.tar.gz to data
Files already downloaded and verified


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 12 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


100%|██████████| 170498071/170498071 [00:01<00:00, 88403498.55it/s]


Extracting data/cifar-10-python.tar.gz to data


...

100%|██████████| 4550980/4550980 [00:00<00:00, 45748385.52it/s]


Extracting data/Imagenet_resize.tar.gz to data


Downloading...
From: https://drive.google.com/uc?id=1Ec-LRSTf6u5vEctKX9vRp9OA6tqnJ0Ay
To: /content/data/places365.zip
100%|██████████| 497M/497M [00:06<00:00, 80.6MB/s]


100%|██████████| 248100043/248100043 [00:05<00:00, 46671787.23it/s]


Extracting data/tiny-imagenet-200.zip to data


In [4]:
model = WideResNet(num_classes=100, pretrained="cifar100-pt").eval().to(device)

detectors = {}
detectors["MSP"] = MaxSoftmax(model)
detectors["Mahalanobis"] = Mahalanobis(model.features)
detectors["ODIN"] = ODIN(model, norm_std=norm_std, eps=0.002)

print(f"- Fitting {len(detectors)} detectors -")
loader_in_train = DataLoader(CIFAR100(root="data", train=True, transform=trans), batch_size=256, num_workers=12)
for name, detector in detectors.items():
    print(f"  > Fitting {name}")
    detector.fit(loader_in_train, device=device)

Downloading: "https://github.com/wetliu/energy_ood/raw/master/CIFAR/snapshots/pretrained/cifar100_wrn_pretrained_epoch_99.pt" to /root/.cache/torch/hub/checkpoints/wrn-cifar100-pt.pt
100%|██████████| 8.66M/8.66M [00:00<00:00, 101MB/s]


- Fitting 3 detectors -
  > Fitting MSP
  > Fitting Mahalanobis
  > Fitting ODIN


In [5]:
results = []

with torch.no_grad():
    for detector_name, detector in detectors.items():
        print(f"- Evaluating {detector_name} -")
        for dataset_name, loader in datasets.items():
            print(f"  {dataset_name}")
            metrics = OODMetrics()
            for x, y in loader:
                metrics.update(detector(x.to(device)), y.to(device))

            r = {"Detector": detector_name, "Dataset": dataset_name}
            r.update(metrics.compute())
            results.append(r)

- Evaluating MSP -
  CIFAR100
  CIFAR10
  MNIST
  FashionMNIST
  LSUNCrop
  LSUNResize
  Textures
  TinyImageNetCrop
  TinyImageNetResize
  Places365
  TinyImageNet
- Evaluating Mahalanobis -
  CIFAR100
  CIFAR10
  MNIST
  FashionMNIST
  LSUNCrop
  LSUNResize
  Textures
  TinyImageNetCrop
  TinyImageNetResize
  Places365
  TinyImageNet
- Evaluating ODIN -
  CIFAR100
  CIFAR10
  MNIST
  FashionMNIST
  LSUNCrop
  LSUNResize
  Textures
  TinyImageNetCrop
  TinyImageNetResize
  Places365
  TinyImageNet


In [7]:
df = pd.DataFrame(results).set_index(['Dataset', 'Detector'])
print(df)
df.to_csv("./result/result.csv")

                                   AUROC   AUPR-IN  AUPR-OUT  FPR95TPR
Dataset            Detector                                           
CIFAR100           MSP          0.344688  0.738764  0.133239    0.9542
CIFAR10            MSP          0.755213  0.924956  0.500790    0.6427
MNIST              MSP          0.746989  0.929356  0.535628    0.5735
FashionMNIST       MSP          0.893815  0.976642  0.737556    0.3661
LSUNCrop           MSP          0.855878  0.843563  0.874041    0.4713
LSUNResize         MSP          0.741252  0.699077  0.773360    0.6496
Textures           MSP          0.735491  0.575050  0.831475    0.7139
TinyImageNetCrop   MSP          0.863217  0.848073  0.882328    0.4335
TinyImageNetResize MSP          0.739807  0.701496  0.767825    0.6605
Places365          MSP          0.739189  0.894454  0.500872    0.6991
TinyImageNet       MSP          0.764159  0.963127  0.327413    0.6524
CIFAR100           Mahalanobis  0.439412  0.803425  0.148923    0.9549
CIFAR1